In [1]:
import pandas as pd
import numpy as np
import requests
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials())
from pprint import pprint

In [2]:
url_1= 'http://www.acclaimedmusic.net/061024/1948-09art.htm'
artists_1= (pd.read_html(url_1,header=0))[0]["Unnamed: 1"]

pages= np.arange(2,6)
artists_list=[]

for page in pages:
    url_2= f"http://www.acclaimedmusic.net/061024/1948-09art{page}.htm"
    artists_df= (pd.read_html(url_2,header=0))
    artists_list.append(artists_df[0]["Unnamed: 1"])

artists= artists_1.append(artists_list)
artists.dropna(inplace=True)
artists_names= artists.reset_index()
artists_names.drop(artists_names.columns[0], axis=1, inplace=True)
artists_names.rename(columns={"Unnamed: 1": "Artist Name"}, inplace=True)
artists_names

,Artist Name
0,The Beatles
1,The Rolling Stones
2,Bob Dylan
3,David Bowie
4,Led Zeppelin
...,...
995,The Sundays
996,Herbert
997,Stephen Stills
998,Mazzy Star


In [3]:
artists_names["Artist ID"]= ""
artists_names.head()

,Artist Name,Artist ID
0,The Beatles,
1,The Rolling Stones,
2,Bob Dylan,
3,David Bowie,
4,Led Zeppelin,


In [4]:
notfoundcount=1

for index, row in artists_names.iterrows():
    name= row['Artist Name']
    results = sp.search(q= name, type='artist')
                 
    
    try:
        artists_names.loc[index,'Artist ID'] = results['artists']["items"][0]["id"]
        artists_names.loc[index,'Artist_Popularity'] = results['artists']["items"][0]["popularity"]      
        artists_names.loc[index,'Artist_Followers'] = results['artists']["items"][0]["followers"]["total"]
        artists_names.loc[index,'Artist Genre']= ",".join(results['artists']["items"][0]["genres"])
             
    except:
        
        print(f"{notfoundcount} ID not found for {row['Artist Name']}")
        artists_names.loc[index,'Artist Name']= np.NaN
        artists_names.loc[index,'Artist ID'] = np.NaN
        artists_names.loc[index,'Artist_Popularity'] = np.NaN
        artists_names.loc[index,'Artist_Followers']= np.NaN
        artists_names.loc[index,'Artist Genre']= np.NaN
        notfoundcount+=1
        pass
        
artists_names.dropna(inplace=True)
artists_names




1 ID not found for Frank Zappa/The Mothers of Invention
2 ID not found for Janis Joplin/Big Brother and The Holding Company
3 ID not found for The Buzzcocks
4 ID not found for Paul McCartney and Wings
5 ID not found for The Small Faces
6 ID not found for The Dead Kennedys
7 ID not found for Harold Melvin and The Bluenotes
8 ID not found for Richard Hell & The Voidoids
9 ID not found for The Mahavishnu Orchestra
10 ID not found for The Nitty Gritty Dirt Band
11 ID not found for The Art Ensemble of Chicago
12 ID not found for Rob Base & DJ E-Z Rock
13 ID not found for Sam the Sham and The Pharaos
14 ID not found for The Screaming Trees
15 ID not found for Michael Hurley/The Unholy Modal Rounders
16 ID not found for Rythim Is Rythim


,Artist Name,Artist ID,Artist_Popularity,Artist_Followers,Artist Genre
0,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,90.0,16410055.0,"beatlesque,british invasion,classic rock,merse..."
1,The Rolling Stones,22bE4uQ6baNwSHPVcDxLCe,84.0,8678847.0,"album rock,british invasion,classic rock,rock"
2,Bob Dylan,74ASZWbe4lXaubB36ztrGX,79.0,4220152.0,"album rock,classic rock,country rock,folk,folk..."
3,David Bowie,0oSGxfWSnnOXhD2fKuz2Gy,82.0,5524310.0,"album rock,art rock,classic rock,dance rock,gl..."
4,Led Zeppelin,36QJpDe2go2KgaRleHCDTp,82.0,9060576.0,"album rock,classic rock,hard rock,rock"
...,...,...,...,...,...
995,The Sundays,4nlqDmbzFzbNITaqjJv7D7,52.0,86662.0,"alternative pop,alternative rock,britpop,dream..."
996,Herbert,5zCaQxjl110XTrm4LQ1CxY,70.0,100778.0,"classical,classical performance,orchestral per..."
997,Stephen Stills,4WlSvDKaq1PA2Nr7cCIPxX,56.0,163305.0,"blues rock,classic rock,country rock,folk,folk..."
998,Mazzy Star,37w38cCSGgKLdayTRjna4W,64.0,410642.0,"alternative rock,dream pop,melancholia,slow core"


In [5]:
artists_names.to_csv("file.csv", index=False,header=True)
artists_names.count()

Artist Name          984
Artist ID            984
Artist_Popularity    984
Artist_Followers     984
Artist Genre         984
dtype: int64

In [6]:
# Adding a column with the first genre for each artist
for index, row in artists_names.iterrows():
    name= row['Artist Name']
    results = sp.search(q= name, type='artist')
    
    try:
        artists_names.loc[index,'Main Genre']= results['artists']["items"][0]["genres"][0]
    except:
        artists_names.loc[index,'Main Genre']= np.NaN
        
        
    

In [13]:
# Validating Counts

artists_names.to_csv("Artistfile.csv", index=False,header=True)
artists_names.count()

Artist Name          984
Artist ID            984
Artist_Popularity    984
Artist_Followers     984
Artist Genre         984
Main Genre           960
dtype: int64

In [8]:
# Create a DataFrame seeking by Artist ID and retrieving the following columns
top10_tracks= pd.DataFrame(columns=["Artist","Artist ID","Tracks","Track ID","Release Date","Track Duration","Popularity"])
top10_tracks


,Artist,Artist ID,Tracks,Track ID,Release Date,Track Duration,Popularity


In [9]:

counter=1
for index, row in artists_names.iterrows():
    artist_id= row['Artist ID']
    artist_top_tracks = sp.artist_top_tracks(artist_id)
     
    track = artist_top_tracks['tracks'][:10]

    for tracks in range(len(track)):    
        try:
            top10_tracks.loc[counter,'Tracks'] = track[tracks]["name"]
            top10_tracks.loc[counter,'Artist'] = track[tracks]["album"]["artists"][0]["name"]
            top10_tracks.loc[counter,'Artist ID'] = track[tracks]["album"]["artists"][0]["id"]
            top10_tracks.loc[counter,'Track ID'] = track[tracks]["id"]
            top10_tracks.loc[counter,'Release Date'] = track[tracks]['album']['release_date']
            top10_tracks.loc[counter,'Track Duration'] = track[tracks]['duration_ms']
            top10_tracks.loc[counter,'Popularity'] = track[tracks]['popularity']
            
            counter+=1
        
        except:
            
            pass
    
top10_tracks    


,Artist,Artist ID,Tracks,Track ID,Release Date,Track Duration,Popularity
1,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Here Comes The Sun - Remastered 2009,6dGnYIeXmHdcikdzNNDMm2,1969-09-26,185733,83
2,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Come Together - Remastered 2009,2EqlS6tkEnglzr7tkKAAYD,1969-09-26,259946,78
3,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Let It Be - Remastered 2009,7iN1s7xHE4ifF5povM6A48,1970-05-08,243026,78
4,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Yesterday - Remastered 2009,3BQHpFgAp4l80e1XslIjNI,1965-08-06,125666,77
5,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Hey Jude - Remastered 2015,0aym2LBJBk9DAYuHHutrIl,2000-11-13,425653,76
...,...,...,...,...,...,...,...
9710,Blind Lemon Jefferson,6EsuMifB8c5rm5P2DdjiLe,Future Blues,7cAXWkbpR8Szb06AFXhAmJ,2013-08-27,180773,15
9711,Blind Lemon Jefferson,6EsuMifB8c5rm5P2DdjiLe,Match Box Blues,4dBUAjMjmAednMqt8PU4AI,1992-01-01,187400,14
9712,Blind Lemon Jefferson,6EsuMifB8c5rm5P2DdjiLe,Rabbit Foot Blues,69xrgBndFd3zs6kqVZ8LDO,1991,179095,13
9713,Blind Lemon Jefferson,6EsuMifB8c5rm5P2DdjiLe,That Black Snake Moan,0I4nvtM87RCvywkmeinsqI,1992-01-01,178640,12


In [14]:
# Adding Audio Features by Track ID
for index, row in top10_tracks.iterrows():
    try:
        track_id = row["Track ID"]
        track_data = sp.audio_features(track_id)
        top10_tracks.loc[index,"Danceability"] = track_data[0]['danceability']
        top10_tracks.loc[index,"Energy"] = track_data[0]['energy']
        top10_tracks.loc[index,"Key"] = track_data[0]['key']
        top10_tracks.loc[index,"Loudness"] = track_data[0]['loudness']
        top10_tracks.loc[index,"Mode"] = track_data[0]['mode']
        top10_tracks.loc[index,"Speechiness"] = track_data[0]['speechiness']
        top10_tracks.loc[index,"Acousticness"] = track_data[0]['acousticness']
        top10_tracks.loc[index,"Instrumentalness"] = track_data[0]['instrumentalness']
        top10_tracks.loc[index,"Liveness"] = track_data[0]['liveness']
        top10_tracks.loc[index,"Valence"] = track_data[0]['valence']
        top10_tracks.loc[index,"Tempo"] = track_data[0]['tempo']
                                                    
        
        
    
        print(f"{index} {row['Track ID']}")
    except:
        top10_tracks.loc[index,"Danceability"]=np.NaN
        top10_tracks.loc[index,"Energy"]= np.NaN
        top10_tracks.loc[index,"Key"]= np.NaN
        top10_tracks.loc[index,"Loudness"]= np.NaN
        top10_tracks.loc[index,"Mode"]= np.NaN
        top10_tracks.loc[index,"Speechiness"] = np.NaN
        top10_tracks.loc[index,"Acousticness"] = np.NaN
        top10_tracks.loc[index,"Instrumentalness"] = np.NaN
        top10_tracks.loc[index,"Liveness"] = np.NaN
        top10_tracks.loc[index,"Valence"] = np.NaN
        top10_tracks.loc[index,"Tempo"] = np.NaN
        
        pass
top10_tracks.head(20)



1 6dGnYIeXmHdcikdzNNDMm2
2 2EqlS6tkEnglzr7tkKAAYD
3 7iN1s7xHE4ifF5povM6A48
4 3BQHpFgAp4l80e1XslIjNI
5 0aym2LBJBk9DAYuHHutrIl
6 5jgFfDIR6FR0gvlA56Nakr
7 4pbG9SUmWIvsROVLF0zF9s
8 5ZBeML7Lf3FMEVviTyvi8l
9 3KfbEIOC7YIv90FIfNSZpo
10 7DD7eSuYSC5xk2ArU62esN
11 63T7DJ1AFDD6Bn8VzG6JE8
12 7HKez549fwJQDzx3zLjHKC
13 2PzU4IB8Dr6mxV3lHuaG34
14 6H3kDe7CGoWYBabAeVWGiD
15 1Ud6moTC0KyXMq1Oxfien0
16 77oU2rjC5XbjQfNe3bD6so
17 52dm9op3rbfAkc1LGXgipW
18 1GcVa4jFySlun4jLSuMhiq
19 61UuPxxYUvacEH6SHIK3sU
20 2aQHcpFfrUoTc9CtxPGotZ
21 6HSXNV0b4M4cLJ7ljgVVeh
22 3AhXZa8sUQht0UEdBJgpGc
23 52vA3CYKZqZVdQnzRrdZt6
24 1lqMLr9Wj7SM2F9AikGcxN
25 18GiV1BaXzPVYpp9rmOg0E
26 4K1imZQQ0yKtJ40vGmUajS
27 2WOjLF83vqjit2Zh4B69V3
28 6Vcwr9tb3ZLO63F8DL8cqu
29 3RkQ3UwOyPqpIiIvGVewuU
30 3y4Uza6K58JXQ7RYya8ZI5
31 7Jh1bpe76CNTCgdgAdBw4Z
32 2EC9IJj7g0mN1Q5VrZkiYY
33 0pQskrTITgmCMyr85tb9qq
34 72Z17vmmeQKAg8bptWvpVG
35 3ZE3wv8V3w2T2f7nOCjV0N
36 3ix6K4wZY29bCujrSznwFZ
37 3gxEZXUjrNbl3TlSrTGbR5
38 6mib3N4E8PZHAGQ3xy7bho
39 0LrwgdLsFaWh9VXIjB

309 5o3DSHw1ORhM3lWxqGRvEf
310 2cKH6blsLnE8oEEmAu34YB
311 5h8ve1B7Gr3k4W0WXAgLxE
312 3whRKAOlJ0M3banzcChvQv
313 6LzjgzpVl0hB3YrBU1b8Z1
314 5U3i59kbTLrxo46TU1FRnF
315 2ivdsb1RcEJjIs5tuHkUC9
316 47Zqlw3jmibtVPy8usPjBj
317 5OcyX5eg7VrJuI833qOCUc
318 44AgaeLQCnvk3PPHIaDeDk
319 00pGV4EInVd77cnOIwPTCv
320 2J1IYUHS1XXyd2NepfTlco
321 29SyMC0plk6qw8NMF7lfRL
322 0UAJH0k4k3slcE83a9UGCe
323 78JmElAFmrPNhLjovDR9Jm
324 0DQqzCHlPyKQXfF8uL01SI
325 5nrmGFJ87crVoJF5xdRqwn
326 7obb4s6A7gf0Lc2AGxodMy
327 7DTfOX4SIuMMt33sSVvLRr
328 1LjPCQHkAYnJbQUgSoHaye
329 5e3isD5st7PGYzSJuoRSIV
330 5HOQNAXcPSwV7Gt8AV8qiP
331 0Puj4YlTm6xNzDDADXHMI9
332 5NLuC70kZQv8q34QyQa1DP
333 5fpizYGbi5IQoEraj6FP0R
334 5qxChyzKLEyoPJ5qGrdurN
335 2tY1gxCKslfXLFpFofYmJQ
336 3gKwVWwKmeuFtPubICbOGc
337 5h40ovzXu7rF7Giwb9oXYy
338 0gmGBCJ5XhOmoNR37MmxEE
339 21REQ1bCUWphT2QK3bLWYQ
340 56H98l4uinRifnTH8e308N
341 4BP3uh0hFLFRb5cjsgLqDh
342 5DnT9a5IM3eMjKgXTWVJvi
343 20OFwXhEXf12DzwXmaV7fj
344 2gE95JskwQ1pCACTpGe1Db
345 36gPq8WG7tDxrblyGVUCiT
3

613 1BPmVDqR4cX9aFC0Qx0eWn
614 1hVRTl4yhWmGW7ImZoO22e
615 6ZgaUtMlUzUf7SqbUUnfbu
616 2JOEPbuwVf0nafGnMSVEhH
617 0RHEzMQT9Vw9GvSdVA5TLC
618 5p95DUZplmVxY0XCQ3hxUV
619 47vmcuvMWFIsMaiHFIGSIu
620 1Rph9i0FWKDa9Aq9H4iiMx
621 4nUGbR53jWii3MSrQGWsBb
622 5y5T6tmz6zPyA0kIUAmZYS
623 0OMNQyneWmmZtTQpULYJcl
624 5G9LvzXcBoIBXOd2jzdJTs
625 3vEmpJQC2p8h0U3OiBufQm
626 5OnyZ56HLhrWOXdzeETqLk
627 4z1fNs2B7KndCsvyPgrhq5
628 0X0LTI0MQG9S3FLSUvjrj5
629 3rrtbnOBDzKQfyYKTezEFe
630 080thMRVMSfE5fem7HBaoF
631 20ZvzoDSefcZo6bj10jgGC
632 0mc51xomEC6CZUZdB8xgQU
633 67Hna13dNDkZvBpTXRIaOJ
634 0j5FJJOmmnXPd0XajFWkMF
635 7uv632EkfwYhXoqf8rhYrg
636 2BndJYJQ17UcEeUFJP5JmY
637 3kKVqFF4pv4EXeQe428zl2
638 7xpw5FSNYXKfdQSBuO0THv
639 46bTgJAQkrkNRxMuFnpRyD
640 1vNNfTgHsrpOXeiaXQBlH7
641 1Rezzt36ybaT2ZbDZpv83D
642 3N2UhXZI4Gf64Ku3cCjz2g
643 0gdmDP6xy3ZV7JNoHWAN9k
644 7MH7TkjQ4RA1vG454dciw0
645 6ALPLbDJ9eWVij9ep3Cgf4
646 250dZY8xYbhHGQpM6bgL52
647 2R5zO3Zi0Of0v1DyxusWbk
648 39PU2z9LesM4HRvD6zY1Tg
649 4svNZgdVZNtpWKbb7nsaLs
6

917 7bOYOD0d7ByMTXvJnQVl8d
918 5Q8SPijJts3lV4fcmCrZXp
919 165NAtd2DRPKNQ6EVY0VG9
920 4v2rkl1mC3zVAz0nXMx9r4
921 5QIbR39hAEDIOkr4ggh4xc
922 6F2vo4sxRNQ58VYe3pdiaL
923 0UFLbtupYWaDGMS4io94Ml
924 2L93TdW2GMue1H2zlkt30F
925 7yzbimr8WVyAtBX3Eg6UL9
926 77K5TB5KZmDZCoJCdd1NvE
927 14CsqOaDkOkrZ49UJLtuOJ
928 5zb7npjQqoJ7Kcpq4yD9qn
929 3mFzIFFFmEXTQs6BDAK2ZZ
930 2DkpoHJ1h88e9dyc6SFIm2
931 5atFAXIgMXP378op4FaxBm
932 60s0QWaOZ2UTzqdIHBCt3x
933 4krhCfJg0znykZoyjeMXRe
934 7LbfuQVct78YoghmoPtsQ8
935 5ZLzl6T8JwqMTMdoE0nCbU
936 1OE5l6sedVcIFELMuxQOPI
937 7M7AwtGvWdMYudqx5Iuh1m
938 3euDGpS2R0NC2Xssqxohva
939 1I6l01BaBgBCVkfQCpIlR6
940 1lNRVjK8MukRZpeurYssIx
941 0mJQlCl9YgxW7kyeltNiVk
942 4DpBfWl3q8e0gGB76lAaox
943 7x2xjJV3YAPeLQJ7u3Kjet
944 16qzGrIMWoxerw2gnW0zuv
945 3EtIraJEHVSbBvLw5msioH
946 0hNVjU6JKydHts0SAjHCno
947 1drLQNS9D5z9lBP4DbtO2R
948 2YkQuh1bijWzQJie9KfFQi
949 3Uh7rAb7F0XGVpEEDwfH1k
950 4XFZey4zmgQV551M0hfaUg
951 4lGUA1a5NCgoRVrvPCrw9G
952 6d01Jkzt0zYiQINWJJFCaf
953 42yexCY4dCftowtAZXuAIj
9

1213 5JIahiVpnzRZWTR9jpjmr0
1214 4fzsfWzRhPawzqhX8Qt9F3
1215 78TTtXnFQPzwqlbtbwqN0y
1216 4Li2WHPkuyCdtmokzW2007
1217 2gZUPNdnz5Y45eiGxpHGSc
1218 1PS1QMdUqOal0ai3Gt7sDQ
1219 2QpGZOhTCHHiKmpSO9FW4h
1220 4S8d14HvHb70ImctNgVzQQ
1221 4EWCNWgDS8707fNSZ1oaA5
1222 22L7bfCiAkJo5xGSQgmiIO
1223 722tgOgdIbNe3BEyLnejw4
1224 3Nf8oGn1okobzjDcFCvT6n
1225 1QaJWSCk3UMKLotnPCIHh1
1226 6Wiamk8BAAP50gAAJopsy2
1227 4cMHCRLPNoEbpnl2rz6GS9
1228 5go7pj7RuuNBsZRaI86hZl
1229 1YhyOTcVjHe73cm6tYrHkA
1230 72B7JwmYZFrf5ub6OZFFVw
1231 3CTxcImER2VN4xGVF4mJCV
1232 32PeRx8ulVvH9kpsIRp7L7
1233 38An8NSITHa1c2NI7TmYAD
1234 3HfEgAaf0koxBpBB8NvGda
1235 52UcjsM15hjCQAUbTW2hy1
1236 4O1j2o3XaZQ1H8xwibgc4q
1237 5KBKqxYY263Tr0haAu3fMz
1238 7ITRNB9OckYIp2SW2iXU8U
1239 41R4fI1Z1w2gsJEFQWdeJs
1240 1kAQmY7yNW6LFdDftDbe1X
1241 0oiGUMBeL8Bn0csxAl4a1W
1242 5NziFO1dYnQPu2EXdTB4UL
1243 1I63q5aYjoyai9CI8QXDrn
1244 4uGIJG1jYFonGc4LGp5uQL
1245 4vpeKl0vMGdAXpZiQB2Dtd
1246 0yLfXULLuRtVj2L4DIrwO7
1247 2zbGXr2XCcRR40R7ymEENc
1248 5eGKna0aflc9wmM

...

8834 1NiFAfPjJukUceg0etgxGC
8835 4Q4w8aRdfhIIEhBututktL
8836 4CsTkNvkwQLel6sxQqrXSj
8837 3hKScGxI28Zor6zJ7JL7aS
8838 1Lpmme6uCsPJIdcbR5aePx
8839 5eGDKrTYl2h6ponZyMMhgZ
8840 3d5Cnswx1dxmjUbn1Dp3JQ
8841 7tZ1sF5HLhqtRO0bITs4lV
8842 6Y19VBGJBoixg6umIkE7ar
8843 4UQXPGa1zyIOsjVP5lhI52
8844 09NRBlKCJmHPedMcOeOz1Y
8845 4B2lJinAkeNLSJjcq3dg8Q
8846 5mb6SzBnxv1ywFSH9V3uxd
8847 3JiiqMYLbnHa1qys1JOpFK
8848 08YEGpKt2LHJ0URCXKHEie
8849 7FCYixd46BlSiO2memrsPo
8850 7w7X5rwDFXwHSqlxtsljP7
8851 3BFXgZr628FqwDP3pQCgvk
8852 3AkxSspcYOvhWTkaMvqyaD
8853 6sxBLLKfLjVaTg5KE4Cy75
8854 4CVP2pvqUxH9tExsHJnzV9
8855 7bp5DfkdK1OAvNJ1U4HfDA
8856 4dmh3OPtAhbEybhGoOG9TE
8857 4qfUlchVD7WM4JhA5xio6m
8858 1OYjATyA5kqLK3CfYfvg3y
8859 4EcwMZC5srOdOVUTIkQWkW
8860 4kxcVF4kEZTQZDlnNfDeur
8861 5mOzvm41Pdh1WOtBKlCtZd
8862 6ohk6OrthqZP3UAgHODmB1
8863 6NTceGKsDLTYehlyRbYp6O
8864 1UTX5wGyy7IC6HeT5MmAJ1
8865 0F845nujLVqCb0XMZCh5Pc
8866 4l9hml2UCnxoNI3yCdL1BW
8867 06UkYE0SZ3Y8zGRFOuWGAe
8868 4gFlHAye7To29Fs4UszQUH
8869 1QlTNF43OyKK9Bo

9128 6wNCdMW82LwJgFrnGqLhpJ
9129 2TY2GcWmTmcE6L40jYC9Xv
9130 77vYwoC7e3pVoPq8BA9CuL
9131 28QF5ELXqDnaBjPjGTlzAK
9132 1crbFuCkGL4kXnAGd63RXq
9133 6TRQPx5Z0YZ0pCQX7JbtlS
9134 6LNwTodkmxsc2dKRJWQr6y
9135 73uEbChpBB29ttwVnwuNVE
9136 3FCN9xduajof2MGIJbWdGe
9137 7kWdf24d03AheJu4KF4mo3
9138 24cOWyNs4aTr4Ol1dtXhh8
9139 0hk1gSyn3wKgdxqF6qaKUZ
9140 1P1I6d4H27mL8eNfPOnGOr
9141 5TEVnlpVfG1gm3KO2X7z1C
9142 0IMxHt3tSiM22EZtDYiNNI
9143 292yiMUvysKNUHCjRRrROQ
9144 6PK9NhUfcPQL5vfkg44dYE
9145 6CryGd6wVcpWf5J5qH7cvZ
9146 6rR6ixVWgJHWIGLWYduP7Y
9147 1SEmfPdFm2xCjkxxJOXxkC
9148 0JZTgwdsRTtCdVA6E0f85S
9149 13pA5RjC7TIxAABdepqSg8
9150 6VPsLr93O6mqmiF5ZfjdxJ
9151 3fjOUeeAionn6XpB7bnqWC
9152 38Ot1P84zo61cmbZFWWOA3
9153 4YWituEPTLy2KZgDTXZUut
9154 2Pjb1ApkiRtEmAGAmPxUnv
9155 1a2iF9XymafjRk56q7oCxo
9156 5XNpdKmlLJPUbwKQceX2tW
9157 5MtN38MGEWJt60LwtBmFNP
9158 0nFZ2HQRnIxiwraCm2X6Uq
9159 7zto61V8ySp03Qi6X1LU2X
9160 1kY6OOTBmRNzhYmu9xSH3a
9161 5MVMARaMoclifNmBtPu0dD
9162 0h4jHdNzUrV9eGR2PT5R8c
9163 1cyftogE58xHzSg

9422 4adwiDSem8c2jgiy66Bcsv
9423 3eUymUfVtcL5J32WxjDBlW
9424 2ypwkRuydYboYjvhduYvWm
9425 4LJcNiq2pBb8OeqrGdBJQw
9426 2EjXfH91m7f8HiJN1yQg97
9427 4ZiDhG5Ift5I6reAEOBBec
9428 5JccvAiwcZ7n3urnXqWPsG
9429 4tMnrDxy7Vmv7HyK3ZTO2w
9430 6en4NMn5Mac3lqJKNxUdaz
9431 60jQTAr2OkFcV4SWSRz3st
9432 78HTdAIfx2UBmFWZN4s7A6
9433 3gshcJd9xIITbbYAJeFEFg
9434 0FgSMiN3cSImUS1tE9EuHH
9435 0XyjtybwqSdqMAFfBEkmZf
9436 2tRI6j8RxFwrxoRKs7dnV8
9437 7nvr4fAv279XAQ7flKOiuT
9438 3n0zZ1V9PXLsxtyTiqDhuQ
9439 3VKly1IGagrEfPAAyw96M4
9440 4DgsvJNNGvV5WPQ40qR3R8
9441 2gnYFQWMzj8bNX4dDJyWzW
9442 2Uw9ZuFXoZgeohnv4zFOq8
9443 2W9pPwFusM5smB8iG4ZA18
9444 2n5MF85CukE7ios8yfqxMC
9445 6UBQONiLH8kNmmB8UrV2Ef
9446 4ovaO5XKOFaHv9fudpt9MF
9447 05M7LRp5Gx7oHxZwdceb2l
9448 5zjZDgjn9jiPBCzZRpxeSw
9449 03xLfJFy99mdr7qhmQNVgB
9450 5v3OV5fmDs89yo1Eub6yFM
9451 54ooygs1H5FVbKRjSeda7c
9452 1weOZ1gOj4KSLRsbFYEQTt
9453 3MFN4eSbi3HGiZ00BatFNX
9454 6MlgLxohy15iSnnrenWKGL
9455 3CtphwpjC0XjIVpLFvGiQR
9456 0W7AbEauB7cP4pidLclApe
9457 4L2S9GbwEgwfSbH

9714 2n1XNQt5Vnfd20qgrhwh9K


,Artist,Artist ID,Tracks,Track ID,Release Date,Track Duration,Popularity,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
1,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Here Comes The Sun - Remastered 2009,6dGnYIeXmHdcikdzNNDMm2,1969-09-26,185733,83,0.557,0.540,9.0,-10.484,1.0,0.0347,0.0339,0.002480,0.1790,0.394,129.171
2,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Come Together - Remastered 2009,2EqlS6tkEnglzr7tkKAAYD,1969-09-26,259946,78,0.533,0.376,9.0,-11.913,0.0,0.0393,0.0302,0.248000,0.0926,0.187,165.007
3,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Let It Be - Remastered 2009,7iN1s7xHE4ifF5povM6A48,1970-05-08,243026,78,0.443,0.403,0.0,-8.339,1.0,0.0322,0.6310,0.000000,0.1110,0.410,143.462
4,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Yesterday - Remastered 2009,3BQHpFgAp4l80e1XslIjNI,1965-08-06,125666,77,0.332,0.179,5.0,-11.830,1.0,0.0326,0.8790,0.000000,0.0886,0.315,96.529
5,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Hey Jude - Remastered 2015,0aym2LBJBk9DAYuHHutrIl,2000-11-13,425653,76,0.386,0.607,10.0,-7.700,1.0,0.0261,0.0112,0.000014,0.0880,0.532,147.207
6,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Blackbird - Remastered 2009,5jgFfDIR6FR0gvlA56Nakr,1968-11-22,138386,75,0.686,0.127,7.0,-14.361,1.0,0.0342,0.7540,0.000014,0.0573,0.372,93.699
7,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,I Want To Hold Your Hand - Remastered 2015,4pbG9SUmWIvsROVLF0zF9s,2000-11-13,145746,74,0.490,0.715,7.0,-5.549,1.0,0.0476,0.3860,0.000000,0.3110,0.866,130.726
8,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Twist And Shout - Remastered 2009,5ZBeML7Lf3FMEVviTyvi8l,1963-03-22,155226,74,0.482,0.849,2.0,-9.198,1.0,0.0452,0.6410,0.000008,0.0414,0.937,124.631
9,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,In My Life - Remastered 2009,3KfbEIOC7YIv90FIfNSZpo,1965-12-03,146333,73,0.688,0.435,9.0,-11.359,1.0,0.0323,0.4490,0.000000,0.1130,0.435,103.239
10,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Help! - Remastered 2009,7DD7eSuYSC5xk2ArU62esN,1965-08-06,139560,72,0.534,0.725,6.0,-7.576,0.0,0.0332,0.1880,0.000000,0.0994,0.763,95.003


In [10]:
top10_tracks.count()


Artist            9714
Artist ID         9714
Tracks            9714
Track ID          9714
Release Date      9714
Track Duration    9714
Popularity        9714
dtype: int64

In [15]:
top10_tracks.to_csv("Trackfile.csv", index=False,header=True)

In [ ]:
#grouped_tracks = top10_tracks.groupby("Artist ID")
# Mean of dancebility vs energy 
# scatter chart 
# linear regression


In [ ]:
#grouped_tracks = top10_tracks.groupby("Artist ID")
# Mean of Valence vs energy 
# scatter chart 
# linear regression


In [ ]:
# how many artist have same genre
# artist with multiple genre are more followed or more popular or not

In [ ]:

# group by each genre and create bins of popularity 10-20 , 20-30, 30-40
# figure out the artist count for each genre and then put the artist into bins of popularity.
# Get the 2 popularity bins and plot for each genere

In [ ]:
# finding out a track with more dancebility is more popular

In [ ]:
# in what decade hat kind of songs were more popular. ( track with what features were more popular)

In [ ]:
# Per decade plot a graph for certain features of tracks to see which feature were more popular in cetrain decades

In [ ]:
# popularity of tracks vs track duration. Are tracks with shorter duration are more popular?
# average Track duration is getting shorter over time